In [92]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re

from keras.models import Model
from keras.layers import Embedding, Dense, Input
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint

!pip install corus
from corus import load_lenta

from collections import Counter

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])"))) - skipping


In [108]:
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout, TimeDistributed, Activation
from tensorflow.keras.callbacks import ModelCheckpoint
# from keras.layers.recurrent import LSTM



from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

#### load and preprocess dataset

In [105]:
# !wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [94]:
lenta_path = 'data/1/lenta-ru-news.csv.gz'
num_rows = 10_000

In [405]:
records = load_lenta(lenta_path)
df = pd.DataFrame([[rec.text, rec.title] for _, rec in zip(range(num_rows), records)], columns=['text', 'title'])

df['title'] = df['title'].apply(lambda x: re.sub("\xa0", " ", x))

reg = re.compile('[^а-яА-Я0-9.,_-_ ]')
df['cleaned_text'] = [reg.sub('', text) for text in df.text.apply(lambda x: x.lower())]
df['cleaned_title'] = [reg.sub('', title) for title in df.title.apply(lambda x: x.lower())]


train_inputs, test_inputs, train_labels, test_labels = train_test_split(df['cleaned_text'].values, df['cleaned_title'].values, test_size=0.1, shuffle=False)

reg = re.compile('[^а-яА-Я_ _-]')
df['tokenized_text'] = [reg.sub(' ', text).split() for text in df.text.apply(lambda x: x.lower())]
df['tokenized_title'] = [reg.sub(' ', title).split() for title in df.title.apply(lambda x: x.lower())]


train_input_tokens, test_input_tokens, train_label_tokens, test_label_tokens = train_test_split(df['tokenized_text'].values, df['tokenized_title'].values, test_size=0.1, shuffle=False)

In [406]:
df.head()

,text,title,cleaned_text,cleaned_title,tokenized_text,tokenized_title
0,Вице-премьер по социальным вопросам Татьяна Го...,Названы регионы России с самой высокой смертно...,вицепремьер по социальным вопросам татьяна гол...,названы регионы россии с самой высокой смертно...,"[вице-премьер, по, социальным, вопросам, татья...","[названы, регионы, россии, с, самой, высокой, ..."
1,Австрийские правоохранительные органы не предс...,Австрия не представила доказательств вины росс...,австрийские правоохранительные органы не предс...,австрия не представила доказательств вины росс...,"[австрийские, правоохранительные, органы, не, ...","[австрия, не, представила, доказательств, вины..."
2,Сотрудники социальной сети Instagram проанализ...,Обнаружено самое счастливое место на планете,сотрудники социальной сети проанализировали п...,обнаружено самое счастливое место на планете,"[сотрудники, социальной, сети, проанализировал...","[обнаружено, самое, счастливое, место, на, пла..."
3,С начала расследования российского вмешательст...,В США раскрыли сумму расходов на расследование...,с начала расследования российского вмешательст...,в сша раскрыли сумму расходов на расследование...,"[с, начала, расследования, российского, вмешат...","[в, сша, раскрыли, сумму, расходов, на, рассле..."
4,Хакерская группировка Anonymous опубликовала н...,Хакеры рассказали о планах Великобритании зами...,хакерская группировка опубликовала новые доку...,хакеры рассказали о планах великобритании зами...,"[хакерская, группировка, опубликовала, новые, ...","[хакеры, рассказали, о, планах, великобритании..."


Build vocab


In [407]:
data = np.hstack(df['tokenized_text'].values)

counter = Counter(data)
count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

words, _ = list(zip(*count_pairs))
word_to_id = dict(zip(words, range(len(words))))

vocabulary = len(word_to_id)

reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

In [409]:
# train_input_ids = [[word_to_id[w] for w in text] for text in train_input_tokens]
# train_label_ids = [[word_to_id[w] for w in text] for text in train_label_tokens]
# test_input_ids = [[word_to_id[w] for w in text] for text in test_input_tokens]
# test_label_ids = [[word_to_id[w] for w in text] for text in test_label_tokens]

sber vocab

In [275]:
ideas = pd.read_csv('/Users/aydar/Desktop/NLP/data/sber_idea/ideas.csv')

In [309]:
reg = re.compile('[^а-яА-Я0-9.,_-_ ]')
ideas['cleaned_text'] = [reg.sub('', text) for text in ideas.content_raw.apply(lambda x: x.lower())]

reg = re.compile('[^а-яА-Я_ ]')
ideas['tokenized_text'] = [reg.sub(' ', text).split() for text in ideas.content_raw.apply(lambda x: x.lower())]

In [315]:
counter_ideas = Counter(np.hstack(ideas.tokenized_text))
count_pairs_ideas = sorted(counter_ideas.items(), key=lambda x: (-x[1], x[0]))

words_ideas, _ = list(zip(*count_pairs_ideas))
word_to_id_ideas = dict(zip(words_ideas, range(len(words_ideas))))

vocabulary_ideas = len(word_to_id_ideas)

reversed_dictionary_ideas = dict(zip(word_to_id_ideas.values(), word_to_id_ideas.keys()))

Build model

In [410]:
hidden_size = 250
num_steps = 20
batch_size = 32
# use_dropout=True

enc_vocabulary = {i:word_to_id[i] for i in word_to_id if word_to_id[i] < 10000}
dec_vocabulary = {i:word_to_id_ideas[i] for i in word_to_id_ideas if word_to_id_ideas[i] < 10000}

In [411]:
config = {'hidden_size': 500,
          'lstm_units': 250,
          'max_input_length':1000,
          'max_target_length': 40, 
          'enc_vocabulary': enc_vocabulary, 
          'dec_vocabulary': dec_vocabulary}


In [412]:
tokenized_text = df.tokenized_text.values[:5000]
tokenized_title = df.tokenized_title.values[:5000]

In [413]:
# transform input

In [418]:
enc_vocabulary, dec_vocabulary = config['enc_vocabulary'], config['dec_vocabulary']

In [426]:
tokenized_text_ids = [[enc_vocabulary[token] if token in enc_vocabulary else -1 for token in text] for text in tokenized_text]
tokenized_title_ids = [[-2] + [dec_vocabulary[token] if token in dec_vocabulary else -1 for token in title] + [-3] for title in tokenized_title]

In [422]:
# [(i,j) for i,j in zip(tokenized_text[0], tokenized_text_ids[0])]

In [431]:
X_padded = pad_sequences(tokenized_text_ids, config['max_input_length'])
y_padded = pad_sequences(tokenized_title_ids, config['max_target_length'])

print(X_padded.shape)
print(y_padded.shape)

(5000, 1000)
(5000, 40)


In [439]:
# sum_unknown_X = [np.mean(np.array(x) == -1) for x in tokenized_text_ids]
# plt.hist(sum_unknown_X)

In [446]:
class Summarizer():
    def __init__(self, config):
        self.max_input_length = config['max_input_length']
        self.max_target_length = config['max_target_length']
        lstm_units = config['lstm_units']
        
        self.enc_vocabulary, self.dec_vocabulary = config['enc_vocabulary'], config['dec_vocabulary']
        
        # encoder
        enc_input = Input(shape=(None,), name='enc_inputs')
        enc_embedding = Embedding(input_dim=len(enc_vocabulary), output_dim=hidden_size)
        enc_lstm = LSTM(lstm_units, return_state=True)#, return_sequences=True
        enc_dense = Dense(hidden_size)
        
        enc_output, enc_state_h, enc_state_c = enc_lstm(enc_embedding(enc_input))
        
        self.encoder_model = Model(enc_input, [enc_output, enc_state_h, enc_state_c], name='Encoder')
        
        
        # training model
        dec_input = Input(shape=(None, hidden_size), name='dec_inputs')
        dec_lstm = LSTM(lstm_units, return_state=True)#, return_sequences=True)
        dec_dense = Dense(len(dec_vocabulary))
        
        
        dec_output, dec_state_h, dec_state_c  = dec_lstm(dec_input, initial_state=[enc_state_h, enc_state_c])
        dec_output = dec_dense(dec_output)
        
        self.model = Model([enc_input, dec_input], dec_output, name='Training')
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        # decoder
        dec_input_states = [Input(shape=lstm_units,), Input(shape=lstm_units,)]
        dec_output, dec_state_h, dec_state_c  = dec_lstm(dec_input, initial_state=dec_input_states)
        
        dec_output = dec_dense(dec_output)
        
        self.decoder_model = Model([dec_input] + dec_input_states, [dec_output] + [dec_state_h, dec_state_c],
                                   name='Decoder')

    def fit(self, X_train, y_train, X_test, y_test, epochs=10, batch_size=32, cache_path='cache_dir/models'):
        
        
        
        checkpoint = ModelCheckpoint(cache_path)
        
        X_train = self.transform_input(X_train)
        X_test = self.transform_input(X_test)
        y_train = self.transform_labels(y_train)
        y_test = self.transform_labels(y_test)
        
        
        
        self.model.fit(X_train, y_train, validation_data=(X_test, y_test),
                      callbacks=[checkpoint])
        
    def transform_input(self, texts):
#         reg = re.compile('[^а-яА-Я_ _-]')
#         tokenized_text = [reg.sub('', text) for text in texts.apply(lambda x: x.lower())]
        tokenized_text = texts
        tokenized_text_ids = [[enc_vocabulary[token] if token in self.enc_vocabulary else -1 for token in text] for text in tokenized_text]
        padded_text_ids = pad_sequences(tokenized_text_ids, self.max_input_length)
        
        return padded_text_ids
        
    def transform_labels(self, titles):
#         reg = re.compile('[^а-яА-Я_ _-]')
#         tokenized_title = [reg.sub(' ', title).split() for title in titles.apply(lambda x: x.lower())]
        tokenized_title = titles
        tokenized_title_ids = [[-2]+[dec_vocabulary[token] if token in self.dec_vocabulary else -1 for token in title]+[-3] for title in tokenized_title]
        padded_title_ids = pad_sequences(tokenized_title_ids, self.max_target_length)
        
        return padded_title_ids
    
    def encode(self, source_tokens):
        encoding, enc_state_h, enc_state_c = self.encoder_model(source_tokens)
        
        return encoding
    
    def decode_sample(self, encoding, hidden_state):
        pass

In [447]:
s = Summarizer(config)

In [449]:
n = 100
s.fit(train_input_ids[:n], train_label_ids[:n], test_input_ids[:n], test_label_ids[:n])

AssertionError: in user code:

    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/aydar/opt/anaconda3/lib/python3.7

In [357]:
s.model.fit()

ValueError: Failed to find data adapter that can handle input: <class 'NoneType'>, <class 'NoneType'>

In [352]:
s.encoder_model.summary()

Model: "Encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
enc_inputs (InputLayer)      [(None, None)]            0         
_________________________________________________________________
embedding_36 (Embedding)     (None, None, 250)         2500000   
_________________________________________________________________
lstm_72 (LSTM)               [(None, 250), (None, 250) 501000    
Total params: 3,001,000
Trainable params: 3,001,000
Non-trainable params: 0
_________________________________________________________________


In [353]:
s.decoder_model.summary()

Model: "Decoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dec_inputs (InputLayer)         [(None, None, 250)]  0                                            
__________________________________________________________________________________________________
input_40 (InputLayer)           [(None, 250)]        0                                            
__________________________________________________________________________________________________
input_41 (InputLayer)           [(None, 250)]        0                                            
__________________________________________________________________________________________________
lstm_73 (LSTM)                  [(None, 250), (None, 501000      dec_inputs[0][0]                 
                                                                 input_40[0][0]             

#### try(

In [104]:

model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
# model.add(LSTM(hidden_size, return_sequences=True))

model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary)))
# model.add(Dense(vocabulary))
model.add(Activation('softmax'))

optimizer = Adam()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 250)           31056750  
_________________________________________________________________
lstm_7 (LSTM)                (None, 20, 250)           501000    
_________________________________________________________________
dropout_38 (Dropout)         (None, 20, 250)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 124227)        31180977  
_________________________________________________________________
activation_11 (Activation)   (None, 20, 124227)        0         
Total params: 62,738,727
Trainable params: 62,738,727
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from keras.utils import to_categorical
class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set
        self.skip_step = skip_step

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                # print(self.current_idx)
                # print(self.num_steps)
                # print(self.data[self.current_idx:self.current_idx + self.num_steps])
                # print(x)
                # print(x[i, :])
                x[i, :] = self.data[self.current_idx:self.current_idx + self.num_steps]
                temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
                # convert all of temp_y into a one hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
            yield x, y

In [ ]:
data_concat = [x for sentence in train_input_ids[:20] for x in sentence]

In [ ]:
train_data_generator = KerasBatchGenerator(data_concat, 
                                           num_steps=num_steps, 
                                           batch_size=1, 
                                           vocabulary=vocabulary,
                                           skip_step=num_steps)
# valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary,
#                                            skip_step=num_steps)

In [ ]:
checkpointer = ModelCheckpoint(filepath='checkpoints/' + '/model-{epoch:02d}.hdf5', verbose=1, save_freq=100)   

In [ ]:
model.fit_generator(train_data_generator.generate(), verbose=True)
# model.fit_generator(train_data_generator.generate(), verbose=True, callbacks=[checkpointer])



     13/Unknown - 9s 716ms/step - loss: 10.8224 - categorical_accuracy: 0.0615

KeyboardInterrupt: ignored

In [ ]:
data = next(train_data_generator.generate())

In [ ]:
prediction = model.predict(data[0])
predict_word = np.argmax(prediction[:, - 2, :])

# true_print_out += reversed_dictionary[test_data[num_steps + i]] + " "
# pred_print_out += reversed_dictionary[predict_word] + " "

In [ ]:
reversed_dictionary[predict_word]

'в'

In [ ]:
res.shape

(32, 20, 124227)

In [ ]:
!mkdir bert_cache
!mkdir bert_cache/transformer

In [ ]:
from transformers import BertTokenizer, TFBertModel
model_name = 'bert-base-multilingual-cased'


cache_dir = "bert_cache/"
cache_dir_models = cache_dir + "bert_models/"
cache_dir_tokenizers = cache_dir + "bert_tokenizers/"
tokenizer = BertTokenizer.from_pretrained(model_name, cache_dir=cache_dir_tokenizers)
bert_model = TFBertModel.from_pretrained(model_name, cache_dir=cache_dir_models)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
bert_tokenized_text = df['tokenized_text'].apply(lambda x: tokenizer.encode(x, max_length=512, truncation=True))
bert_tokenized_title = df['tokenized_title'].apply(lambda x: tokenizer.encode(x, max_length=32, truncation=True))


input_ids = pad_sequences(bert_tokenized_text, maxlen=512, truncating="post", padding="post")
label_ids = pad_sequences(bert_tokenized_title, maxlen=32, truncating="post", padding="post")

bert_text_train, bert_text_test, bert_title_train, bert_title_test = train_test_split(input_ids, label_ids, test_size=0.1)

In [ ]:
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers.recurrent import LSTM

from keras.preprocessing.sequence import pad_sequences



In [ ]:
# input_layer = Input(shape=(512), dtype='int64')
# bert = bert_model.bert(input_layer)
# lstm = LSTM(units=250, return_state=True)(bert[0])
# classifier = Dense(units=2000, activation='softmax')(lstm[0])
# model = Model(inputs=input_layer, outputs=classifier)

# model.layers[1].trainable = False
# model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       ((None, 512, 768), (None, 177853440 
_________________________________________________________________
lstm_3 (LSTM)                [(None, 250), (None, 250) 1019000   
_________________________________________________________________
dense_5 (Dense)              (None, 2000)              502000    
Total params: 179,374,440
Trainable params: 1,521,000
Non-trainable params: 177,853,440
_________________________________________________________________


In [ ]:
input_layer = Input(shape=(512), dtype='int64')
bert = bert_model.bert(input_layer)
classifier = Dense(units=119547, activation='softmax')(bert[0])
model = Model(inputs=input_layer, outputs=classifier)
model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       ((None, 512, 768), (None, 177853440 
_________________________________________________________________
dense_6 (Dense)              (None, 512, 119547)       91931643  
Total params: 269,785,083
Trainable params: 91,931,643
Non-trainable params: 177,853,440
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'loss'])

In [ ]:
bert_text_train.shape

(900, 512)

In [ ]:
bert_title_train.shape

(900, 32)

In [ ]:
model(bert_text_train)

In [ ]:
model.fit(bert_text_train, bert_title_train, batch_size=32)

ValueError: in user code:

    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/aydar/opt/anaconda3/lib/python3.7/sit

In [ ]:
checkpointer = ModelCheckpoint(filepath='checkpoints/' + '/model-{epoch:02d}.hdf5', verbose=1)   

In [ ]:
# model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
#                         validation_data=valid_data_generator.generate(),
#                         validation_steps=len(valid_data)//(batch_size*num_steps), callbacks=[checkpointer])

In [ ]:
model.fit()

#### from keras-text-summarization

In [ ]:
from collections import Counter

MAX_INPUT_SEQ_LENGTH = 1500
MAX_TARGET_SEQ_LENGTH = 50
MAX_INPUT_VOCAB_SIZE = 5000
MAX_TARGET_VOCAB_SIZE = 2000

In [ ]:
# def fit_text(X, Y, input_seq_max_length=None, target_seq_max_length=None):
X = train_input_tokens
Y = train_label_tokens
input_seq_max_length=None
target_seq_max_length=None


if input_seq_max_length is None:
    input_seq_max_length = MAX_INPUT_SEQ_LENGTH
if target_seq_max_length is None:
    target_seq_max_length = MAX_TARGET_SEQ_LENGTH
input_counter = Counter()
target_counter = Counter()
max_input_seq_length = 0
max_target_seq_length = 0

for line in X:
    text = [word.lower() for word in line.split(' ')]
    seq_length = len(text)
    if seq_length > input_seq_max_length:
        text = text[0:input_seq_max_length]
        seq_length = len(text)
    for word in text:
        input_counter[word] += 1
    max_input_seq_length = max(max_input_seq_length, seq_length)

for line in Y:
    line2 = 'START ' + line.lower() + ' END'
    text = [word for word in line2.split(' ')]
    seq_length = len(text)
    if seq_length > target_seq_max_length:
        text = text[0:target_seq_max_length]
        seq_length = len(text)
    for word in text:
        target_counter[word] += 1
        max_target_seq_length = max(max_target_seq_length, seq_length)

input_word2idx = dict()
for idx, word in enumerate(input_counter.most_common(MAX_INPUT_VOCAB_SIZE)):
    input_word2idx[word[0]] = idx + 2
input_word2idx['PAD'] = 0
input_word2idx['UNK'] = 1
input_idx2word = dict([(idx, word) for word, idx in input_word2idx.items()])

target_word2idx = dict()
for idx, word in enumerate(target_counter.most_common(MAX_TARGET_VOCAB_SIZE)):
    target_word2idx[word[0]] = idx + 1
target_word2idx['UNK'] = 0

target_idx2word = dict([(idx, word) for word, idx in target_word2idx.items()])

num_input_tokens = len(input_word2idx)
num_target_tokens = len(target_word2idx)

config = dict()
config['input_word2idx'] = input_word2idx
config['input_idx2word'] = input_idx2word
config['target_word2idx'] = target_word2idx
config['target_idx2word'] = target_idx2word
config['num_input_tokens'] = num_input_tokens
config['num_target_tokens'] = num_target_tokens
config['max_input_seq_length'] = max_input_seq_length
config['max_target_seq_length'] = max_target_seq_length

# return config

In [ ]:
fit_text

In [371]:
def transform_input_text(texts, max_input_seq_length=1000):
#     input_word2idx = {}
    temp = []
    for line in texts:
        x = []
        for word in line.lower().split(' '):
            wid = 1
            if word in word_to_id:
                wid = word_to_id[word]
            x.append(wid)
            if len(x) >= max_input_seq_length:
                break
        temp.append(x)
    temp = pad_sequences(temp, maxlen=max_input_seq_length)

    print(temp.shape)
    return temp, word_to_id

def transform_target_encoding(texts, max_target_seq_length=40):
    temp = []
    for line in texts:
        x = []
        line2 = 'START ' + line.lower() + ' END'
        for word in line2.split(' '):
            x.append(word)
            if len(x) >= max_target_seq_length:
                break
        temp.append(x)

    temp = np.array(temp)
    print(temp.shape)
    return temp

In [372]:
transform_input_text([' '.join(train_input_tokens[0])])

(1, 1000)


NameError: name 'input_word2idx' is not defined

'вицепремьер по социальным вопросам татьяна голикова рассказала в каких регионах россии зафиксирована наиболее высокая смертность от рака сообщает риа новости по словам голиковой чаще всего онкологические заболевания становились причиной смерти в псковской тверской тульской и орловской областях а также в севастополе вицепремьер напомнила что главные факторы смертности в россии  рак и болезни системы кровообращения в начале года стало известно что смертность от онкологических заболеваний среди россиян снизилась впервые за три года по данным росстата в  году от рака умерли  тысяч человек это на  процента меньше чем годом ранее'

In [ ]:
res = transform_input_text(train_input_tokens)

(900, 1000)


In [ ]:
res[0][-2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

#### calculate variables

In [ ]:
HIDDEN_UNITS = 100
LOAD_WEIGHTS = False

In [ ]:
from collections import Counter

In [ ]:
MAX_INPUT_SEQ_LENGTH = 500
MAX_TARGET_SEQ_LENGTH = 50
MAX_INPUT_VOCAB_SIZE = 50000
MAX_TARGET_VOCAB_SIZE = 10000

1245.0

In [ ]:
num_input_tokens = int(np.median([len(x) for x in train_input_tokens]))
max_input_seq_length = 1500

num_target_tokens = int(np.median([len(x) for x in train_label_tokens]))
max_output_seq_length = 50

In [ ]:
encoder_inputs = Input(shape=(None,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=num_input_tokens, output_dim=HIDDEN_UNITS,
                              input_length=max_input_seq_length, name='encoder_embedding')

encoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, name='encoder_lstm')

encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding(encoder_inputs))
encoder_states = [encoder_state_h, encoder_state_c]

In [ ]:
decoder_inputs = Input(shape=(None, num_target_tokens), name='decoder_inputs')
decoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, name='decoder_lstm')
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs,
                                                                 initial_state=encoder_states)
decoder_dense = Dense(units=num_target_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

if LOAD_WEIGHTS:
    model.load_weights(weight_file_path)
    
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_inputs = [Input(shape=(HIDDEN_UNITS,)), Input(shape=(HIDDEN_UNITS,))]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

In [ ]:
model.fit()

ValueError: Failed to find data adapter that can handle input: <class 'NoneType'>, <class 'NoneType'>

In [ ]:
with open('log.txt', 'a') as log:
    try:
        for cycle_num in range(num_cycles):
            print(cycle_num)

            history = summarizer.fit(train_input_tokens, train_label_tokens, test_input_tokens, test_label_tokens, epochs=num_epochs)


            # log results
            log.write('Epoch number: {}'.format(str((cycle_num+1) * num_epochs )))
            summarizer.model.save_weights('weights_cycle_{}.h5'.format(cycle_num))
            plot_and_save_history(history, summarizer.model_name, '_result_images/plot_seq2seq_cycle_{}.png'.format(cycle_num), metrics={'loss'})

            for i in np.random.permutation(np.arange(len(train_input_tokens)))[0:20]:
                x = train_input_tokens[i]
                actual_headline = train_label_tokens[i]
                headline = summarizer.summarize(x)
                log.write('Generated Headline: ' + str(headline))
                log.write('Original Headline: ' + str(actual_headline))

            log.write('\n\n')
    except KeyboardInterrupt:
        summarizer.model.save_weights('weights_interrupt.h5')

In [ ]:
sd